In [1]:
import sys
sys.path.append('/home/macul/libraries/MXNet2Caffe')
sys.path.append('/home/macul/insightface/src/')
sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/mxiter')
sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/mxconv')
import mxnet as mx
import caffe
from collections import namedtuple
import numpy as np
import cv2

ImportError: No module named mxnet

In [ ]:
# backup original .json file

In [2]:
mxModelPath = '/media/macul/black/mxnet_training/r50/server_train16/train_16'
mxModelEpoch = [39,40,41,42]

In [7]:
sym_tmp, arg_params, aux_params = mx.model.load_checkpoint(mxModelPath, mxModelEpoch[0])
all_layers = sym_tmp.get_internals()
output_list = all_layers.list_outputs()
sym3 = all_layers['embedding_output']
model = mx.mod.Module(symbol=sym3, label_names=None, context=mx.cpu())
model.bind(for_training=False, data_shapes=[('data', (1,3,112,112))])


In [3]:
for i in range(1,len(mxModelEpoch)):
    _, arg_p, aux_p = mx.model.load_checkpoint(mxModelPath, mxModelEpoch[i])
    for k in arg_params.keys():
        arg_params[k] += arg_p[k]
    for k in aux_params.keys():
        aux_params[k] += aux_p[k]

for k in arg_params.keys():
    arg_params[k] /= len(mxModelEpoch)


for k in aux_params.keys():
    aux_params[k] /= len(mxModelEpoch)


In [8]:
model.set_params(arg_params, aux_params, allow_missing=False, allow_extra=False)

In [10]:
model.save_checkpoint(mxModelPath, mxModelEpoch[0]*100+mxModelEpoch[-1])

In [ ]:
# restore original .json file